# Project

---

## Downloading Data

In this project, the attention is not directed towards attributes such as 'smiling' or 'glasses' from the CelebA labels, instead, we’re interested in the identity of datapoints. So, we have chosen to work with 1,000 unique identities. For these identities, we have 27,018 training images and 3,009 test images. You can find the images and their corresponding labels in `./data/trainset.txt` for training data and `./data/testset.txt` for test data.

Furthermore, there is a distinct dataset available in `./data/ganset.txt` which is public and is utilized to pretrain the GAN. It's crucial to note that this dataset does not have class overlap with the private data designated for training the classifier, ensuring that the identities in the GAN pretraining set are exclusive from those in the classifier’s training and test sets.

to download the CelebA dataset run the cell below.

> ### NOTE
> There are occasions when downloading the CelebA dataset using `torchvision`
> results in a Google Drive error. This is due to the dataset being hosted on
> Google Drive, which sometimes restricts the number of download requests.
>
> In such cases, an alternative is to download the CelebA dataset directly 
> from Kaggle using the following link:
> [CelebA Dataset on Kaggle](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset)
> Once downloaded, the CelebA dataset should be added to the `./data` directory


In [ ]:
%run data_downloader.py

---

## Training a classifier

Here is where we train the target model classifier. We choose 3 sophisticated pre-trained models that have been trained on ImageNet data and then fine-tune them to classify our celebrity identities data. Details on these architectures can be found below. Training details can be found and altered at `./config/classify.json`

| Model       | Size (MB) | Parameters | Depth | Time (ms) per inference step (CPU) | Time (ms) per inference step (GPU) |
|-------------|-----------|------------|-------|------------------------------------|------------------------------------|
| Resnet152   | 232       | 60.4M      | 311   | 127.4                              | 6.5                                |
| VGG16       | 528       | 138.4M     | 16    | 69.5                               | 4.2                                |
| FaceNet64   | 98        | 25.6M      | 22    | 58.2                               | 4.6                                |


> ## Note
> You can find pretrained checkpoints of these model backbones and also fune-tuned checkpoints [here](https://drive.google.com/drive/folders/1ZTTrRJr-2HOgfyxndP8a9R2Hb_UOgV6J).

### Train a VGG16 based classifier

Running the cell below will train a classifier based on the *VGG16* model.

In [ ]:
%run train_classifier.py --model_name "VGG16"

### Train a FaceNet based classifier

Running the cell below will train a classifier based on the *FaceNet* model.

> to train this model you need to download `backbone_ir50_ms1m_epoch120.pth` from [here](https://drive.google.com/drive/folders/1ZTTrRJr-2HOgfyxndP8a9R2Hb_UOgV6J) and add it to `./target_model`

In [ ]:
%run train_classifier.py --model_name "FaceNet"

### Train an IR152 based classifier

Running the cell below will train a classifier based on the *ResNet-152* model.

> to train this model you need to download `Backbone_IR_152_Epoch_112_Batch_2547328_Time_2019-07-13-02-59_checkpoint.pth` from [here](https://drive.google.com/drive/folders/1ZTTrRJr-2HOgfyxndP8a9R2Hb_UOgV6J) and add it to `./target_model`

In [ ]:
%run train_classifier.py --model_name "IR152"

---

## Training General Binary GAN

Executing the cell below will initiate the training of a general binary GAN, excluding any enhancements mentioned in the paper.. A pretrained checkpoint for this GAN can be accessed [here.](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing)

In [ ]:
%run k+1_gan.pyinary_gan.py

---

## Training an Inversion-Specifi GAN

Executing the cell will initialize an Inversion-Specific GAN. This is the model proposed in [2], which is the article at the center of my assignment. The --model_name_T flag indicates the target model being attacked here. A pre-trained checkpoint for this GAN can be accessed [here.](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing) Additionally, model checkpoints and generated image results are saved in the `./improvedGAN` directory.

### Train a GAN VGG16

Train an inversion-specific GAN that attacks VGG16 model.

> #### Important Note
For training, ensure that the target model, named `VGG16_86.30_allclass.tar`, obtained either from the previous stage or [here](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing), is placed at `./target_model/target_ckp`.

In [ ]:
%run k+1_gan.py --model_name_T "VGG16"

### Attack FaceNet64

Train an inversion-specific GAN that attacks FaceNet64 model.

> #### Important Note
For training, ensure that the target model, named `FaceNet64_88.50.tar`, obtained either from the previous stage or [here](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing), is placed at `./target_model/target_ckp`.

In [ ]:
%run k+1_gan.py --model_name_T "FaceNet64"

### Attack IR152

Train an inversion-specific GAN that attacks ResNet-152 model.

> #### Important Note
For training, ensure that the target model, named `IR152_91.16.tar`, obtained either from the previous stage or [here](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing), is placed at `./target_model/target_ckp`.

In [ ]:
%run k+1_gan.py --model_name_T "IR152"

---

## Accuracy and Top 5 Accuracy


This section of the code is where the actual attack is executed. In each cell, one model is selected as the target and subsequently attacked. 

- The `improved_flag` parameter indicates whether the 'Inversion-Specific GAN', as mentioned in the paper, is being used for the attack.
- The `dist_flag` parameter signifies whether distribution recovery is employed or not.

If both `improved_flag` and `dist_flag` are set to false, simply the attack is GMI as proposed in felan.

We will conduct experiments utilizing various configurations of these flags to ascertain their impact and assess whether their presence influences the results.

> #### Important Note
> Before executing the cells below, ensure that the model `FaceNet_95.88.tar`, downloaded from [here](https://drive.google.com/drive/folders/1eCuJXdpKlrIAf9jIYxQ1cHviCQ4hxL--?usp=sharing), is located at `./target_model/target_ckp`. This model serves as the Evaluation Classifier. The role of the Evaluation Classifier is explained in the report file.

For each attack, we will examine all possible combinations of flags and compare the results, which can be found in the project report.

### Attack VGG16 Bases Classifier

In [ ]:
%run recovery.py --model "VGG16" --improved_flag --dist_flag

In [ ]:
%run recovery.py --model "VGG16" --improved_flag 

In [ ]:
%run recovery.py --model "VGG16" --dist_flag

In [ ]:
%run recovery.py --model "VGG16" 

### Attack VGG16 FaceNet Classifier

In [ ]:
%run recovery.py --model "FaceNet64" --improved_flag --dist_flag

In [ ]:
%run recovery.py --model "FaceNet64" --improved_flag 

In [ ]:
%run recovery.py --model "FaceNet64" --dist_flag

In [ ]:
%run recovery.py --model "FaceNet64"

### Attack IR152 Bases Classifier

In [ ]:
%run recovery.py --model "IR152" --improved_flag --dist_flag

In [ ]:
%run recovery.py --model "IR152" --improved_flag

In [ ]:
%run recovery.py --model "IR152" --dist_flag

In [ ]:
%run recovery.py --model "IR152"

---

## Calculating the Fréchet inception distance (FID)

FID (Fréchet Inception Distance) is a widely recognized method used to evaluate the performance of GANs. In this project, since the original code provided by the authors didn't include a module to calculate FID, i implemented it separately. By executing the provided code, we will be able to calculate the FID for both the GMI model and the Inversion-Specific model to thoroughly analyze their performances.

### Calculate FID for GMI

In [ ]:
%run calculate_FID.py

### Calculate FID for Inversion-Specific GAN

In [ ]:
%run calculate_FID.py --improved_flag